In [7]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [8]:
from pathlib import Path
import sys
sys.path.append(str(Path.cwd() / "_scripts"))

In [9]:
from shapely import Polygon
from functools import partial


from case_edits.ezcase_new import *
from new_subsurfaces.preparation import *
from geometry.wall_normal import WallNormal


In [10]:

inputs_dir = "case_red_d3"
outputs_dir = "tests/test22_svg2plan"

case_path = get_case_inputs_path(inputs_dir)
idf = create_ezcase(outputs_dir, inputs_dir)



width: 0.57 NOT GREATER THAN 0.81
Editing (2, 6) 

width: 0.66 NOT GREATER THAN 0.81
Editing (2, 7) 

width: 0.71 NOT GREATER THAN 0.81
Editing (6, 4) 

width: 0.71 NOT GREATER THAN 0.81
Editing (3, 6) 



In [11]:
dom = create_domain_for_rectangular_wall(get_zone_walls(idf, 4)[0])
create_nine_points_for_domain(dom)

{'top_left': PlacementPoint(point=Coord(x=1.8135000000000001, y=3.0195), functions=[<function push_down at 0x10f560670>]),
 'top_middle': PlacementPoint(point=Coord(x=2.475, y=3.0195), functions=[<function push_down at 0x10f560670>, <function push_left_half at 0x10f560b80>]),
 'top_right': PlacementPoint(point=Coord(x=3.1365, y=3.0195), functions=[<function push_down at 0x10f560670>, <function push_left at 0x10f560a60>]),
 'middle_left': PlacementPoint(point=Coord(x=1.8135000000000001, y=1.525), functions=[<function push_down_half at 0x10f560af0>]),
 'middle_middle': PlacementPoint(point=Coord(x=2.475, y=1.525), functions=[<function push_down_half at 0x10f560af0>, <function push_left_half at 0x10f560b80>]),
 'middle_right': PlacementPoint(point=Coord(x=3.1365, y=1.525), functions=[<function push_down_half at 0x10f560af0>, <function push_left at 0x10f560a60>]),
 'bottom_left': PlacementPoint(point=Coord(x=1.8135000000000001, y=0.0305), functions=[]),
 'bottom_middle': PlacementPoint(poi

In [12]:
idf.idfobjects["WINDOW"]

[
WINDOW,
    Block 04 Storey 0 Wall 0001 Window,    !- Name
    ,                         !- Construction Name
    Block 04 Storey 0 Wall 0001,    !- Building Surface Name
    ,                         !- Frame and Divider Name
    1.0,                      !- Multiplier
    2.12,                     !- Starting X Coordinate
    0.0305,                   !- Starting Z Coordinate
    0.71,                     !- Length
    1.52;                     !- Height
, 
WINDOW,
    Block 04 Storey 0 Wall 0002 Window,    !- Name
    ,                         !- Construction Name
    Block 04 Storey 0 Wall 0002,    !- Building Surface Name
    ,                         !- Frame and Divider Name
    1.0,                      !- Multiplier
    2.71,                     !- Starting X Coordinate
    0.0305,                   !- Starting Z Coordinate
    0.71,                     !- Length
    1.52;                     !- Height
, 
WINDOW,
    Block 00 Storey 0 Wall 0003 Window,    !- Name
    ,      

In [9]:
# path_to_inputs = get_case_inputs_path(inputs_dir)
# case = initialize_case(outputs_dir)
# case.idf = add_rooms(case.idf, path_to_inputs)
# pairs = get_subsurface_pairs_from_case(case_path)

In [8]:
case.idf

In [16]:
from new_subsurfaces.creator import SubsurfaceCreator


sc = SubsurfaceCreator(case.idf, pairs[0])


In [19]:
sc.get_surface()
sc.get_start_location()
sc.gather_details()
sc.initialize_object()
sc.update_attributes()

In [20]:
sc.obj0


DOOR:INTERZONE,
    Block 01 Storey 0 Wall 0001_1 Door,    !- Name
    ,                         !- Construction Name
    Block 01 Storey 0 Wall 0001_1,    !- Building Surface Name
    ,                         !- Outside Boundary Condition Object
    1.0,                      !- Multiplier
    0.4849999998975545,       !- Starting X Coordinate
    3.050000e-02,             !- Starting Z Coordinate
    0.81,                     !- Length
    2.03;                     !- Height

In [25]:
sc.obj0.Building_Surface_Name

''

In [10]:
get_zone_walls(case.idf, 1)[0]


BUILDINGSURFACE:DETAILED,
    Block 01 Storey 0 Wall 0003,    !- Name
    wall,                     !- Surface Type
    Project Wall,             !- Construction Name
    Block 01 Storey 0,        !- Zone Name
    ,                         !- Space Name
    outdoors,                 !- Outside Boundary Condition
    ,                         !- Outside Boundary Condition Object
    SunExposed,               !- Sun Exposure
    WindExposed,              !- Wind Exposure
    autocalculate,            !- View Factor to Ground
    autocalculate,            !- Number of Vertices
    0,                        !- Vertex 1 Xcoordinate
    1.78,                     !- Vertex 1 Ycoordinate
    3.05,                     !- Vertex 1 Zcoordinate
    0,                        !- Vertex 2 Xcoordinate
    1.78,                     !- Vertex 2 Ycoordinate
    0,                        !- Vertex 2 Zcoordinate
    0,                        !- Vertex 3 Xcoordinate
    0,                        !- Vertex 

In [5]:
print(f"{0.00045678:.2f}")

0.00


In [6]:
for p in pairs:
    surf, pair = get_approp_surface_and_attrs(idf, p)

width: 0.57 NOT GREATER THAN 0.81
Editing (2, 6) 

width: 0.66 NOT GREATER THAN 0.81
Editing (2, 7) 

width: 0.71 NOT GREATER THAN 0.81
Editing (6, 4) 

width: 0.71 NOT GREATER THAN 0.81
Editing (3, 6) 



In [15]:
surfs = [i for i in  get_zone_walls(idf,0)]
[(i.Name, WallNormal(i.azimuth)) for i in  get_zone_walls(idf,0)]

[('Block 00 Storey 0 Wall 0002', <WallNormal.NORTH: 0.0>),
 ('Block 00 Storey 0 Wall 0003', <WallNormal.WEST: 270.0>),
 ('Block 00 Storey 0 Wall 0001_1', <WallNormal.EAST: 90.0>),
 ('Block 00 Storey 0 Wall 0001_2', <WallNormal.EAST: 90.0>),
 ('Block 00 Storey 0 Wall 0004_1', <WallNormal.SOUTH: 180.0>)]

In [17]:
north_wall = surfs[0]
west_wall = surfs[1]

In [20]:
surfs[2]


BUILDINGSURFACE:DETAILED,
    Block 00 Storey 0 Wall 0001_1,    !- Name
    wall,                     !- Surface Type
    Project Partition,        !- Construction Name
    Block 00 Storey 0,        !- Zone Name
    ,                         !- Space Name
    surface,                  !- Outside Boundary Condition
    Block 02 Storey 0 Wall 0003_1,    !- Outside Boundary Condition Object
    NoSun,                    !- Sun Exposure
    NoWind,                   !- Wind Exposure
    autocalculate,            !- View Factor to Ground
    autocalculate,            !- Number of Vertices
    1.7,                      !- Vertex 1 Xcoordinate
    1.779999999795109,        !- Vertex 1 Ycoordinate
    3.0499999998137355,       !- Vertex 1 Zcoordinate
    1.7,                      !- Vertex 2 Xcoordinate
    1.779999999795109,        !- Vertex 2 Ycoordinate
    0,                        !- Vertex 2 Zcoordinate
    1.7,                      !- Vertex 3 Xcoordinate
    1.7999999998137355,       

In [14]:
from new_subsurfaces.ep_helpers import create_domain_for_rectangular_wall


d = create_domain_for_rectangular_wall(west_wall)
d

Domain(width=Range(min=1.78, max=3.15), height=Range(min=0.0, max=3.05))

In [18]:
d.width.buffered_min(0.01)

1.7937

In [22]:
BUFFER = 0.01


In [9]:
cs = [Coordinate3D(*i) for i in north_wall.coords]
cs

[Coordinates(x=1.7, y=3.15, z=3.05),
 Coordinates(x=1.7, y=3.15, z=0.0),
 Coordinates(x=0.0, y=3.15, z=0.0),
 Coordinates(x=0.0, y=3.15, z=3.05)]

In [14]:
xs = sorted(set([i.x for i in cs]))
xs

[0.0, 1.7]

In [17]:
ys = sorted(set([i.y for i in cs]))
ys

[3.15]

In [18]:
width = Range(*ys) if len(ys) > 1 else Range(*xs)
width

Range(min=0.0, max=1.7)

0.0